In [3]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import matplotlib.pyplot as plt 

urls=["https://www.news18.com/news/auto/road-safety-this-country-tops-list-of-worlds-worst-drivers-india-ranks-at-4th-place-7141687.html"]
headers_={'user-agent':'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.0)'}

for url in urls:
  page=requests.get(url, headers=headers_)

  soup=bs(page.content,'html.parser')

  Contents=soup.findAll("p")

Scrapped_data=[]
for content in Contents:
  Text=content.get_text()
  Scrapped_data.append([Text])

df=pd.DataFrame(Scrapped_data,columns=["Text"])

display(df)

df.to_json("accident.json")

,Text
0,By: Auto Desk
1,Edited By: Mayank Gupta
2,Trending Desk
3,"Last Updated: February 28, 2023, 14:22 IST"
4,"New Delhi, India"
5,Image Used for Representational Purpose (Image...
6,Indian roads have long been dominated by heavy...
7,Also Read: Passengers at Terminal 3 of Delhi A...
8,"To make the list, the organisation studied dri..."
9,India ranked the lowest on social media sentim...


In [5]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
import nltk
from nltk.corpus import stopwords

#Vis
import pyLDAvis
import pyLDAvis.gensim

import warnings 
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [9]:
def load_data(file):
  with open(file,"r",encoding="utf-8") as f:
    data=json.load(f)
  return(data)

def write_data(file,data):
  with open(file,"w",encoding="utf-8") as f:
    json.dump(data,f,indent=4)

In [11]:

nltk.download("stopwords")
print(stopwords)


<WordListCorpusReader in '.../corpora/stopwords' (not loaded yet)>


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
data=load_data("/content/accident.json")["Text"]
data=data["8"]
data=data.split(".")
data

['To make the list, the organisation studied drivers in more than 50 nations and ranked them based on parameters such as road quality, traffic index, speed limits, deaths due to road accident injuries, and even legal blood alcohol levels',
 ' Thailand was placed first on the list of countries with the worst drivers',
 ' This Southeast Asian nation was followed by Peru, Lebanon, and then India',
 '']

In [15]:
def lemmatization(texts,allowed_postages=["NOUN","ADJ","VERB","ADV"]):
  nlp=spacy.load("en_core_web_sm",disable=["senter","ner","parser"])
  texts_out=[]
  for text in texts:
    doc=nlp(text)
    new_text=[]
    for token in doc:
      if token.pos_ in allowed_postages:
        new_text.append(token.lemma_)
    final=" ".join(new_text)
    texts_out.append(final)
  return (texts_out)

lemmatized_texts=lemmatization(data)
lemmatized_texts

['make list organisation study driver more nation rank base parameter such road quality traffic index speed limit death road accident injury even legal blood alcohol level',
 'place first list country bad driver',
 'southeast asian nation follow then',
 '']

In [17]:
def gen_words(texts):
  final=[]
  for text in texts:
    new=gensim.utils.simple_preprocess(text,deacc=True)
    final.append(new)
  return(final)

data_words=gen_words(lemmatized_texts)
print(data_words)

[['make', 'list', 'organisation', 'study', 'driver', 'more', 'nation', 'rank', 'base', 'parameter', 'such', 'road', 'quality', 'traffic', 'index', 'speed', 'limit', 'death', 'road', 'accident', 'injury', 'even', 'legal', 'blood', 'alcohol', 'level'], ['place', 'first', 'list', 'country', 'bad', 'driver'], ['southeast', 'asian', 'nation', 'follow', 'then'], []]


In [22]:
id2word=corpora.Dictionary(data_words)
corpus=[]
for text in data_words:
  new = id2word.doc2bow(text)
  corpus.append(new)

print(corpus)

word = id2word
print(word)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1)], [(5, 1), (12, 1), (25, 1), (26, 1), (27, 1), (28, 1)], [(15, 1), (29, 1), (30, 1), (31, 1), (32, 1)], []]
Dictionary(33 unique tokens: ['accident', 'alcohol', 'base', 'blood', 'death']...)


In [24]:
lda_model= gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=30,random_state=100,update_every=1,
                                           chunksize=100,passes=10,alpha="auto")

In [25]:
pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim.prepare(lda_model,corpus,id2word,mds="mmds",R=30)
vis

/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/dist-packages/sklearn/manifold/_mds.py:299: FutureWarning: The default value of `normalized_stress` will change to `'auto'` in version 1.4. To suppress this warning, manually set the value of `normalized_stress`.
  warnings.warn(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.031006 -0.079083       1        1  68.516039
12     0.248654  0.091814       2        1  14.649687
21    -0.238955  0.123318       3        1  11.995842
6     -0.000826 -0.004687       4        1   0.214923
0     -0.001534 -0.005052       5        1   0.177827
16    -0.001534 -0.005052       6        1   0.177827
28    -0.001534 -0.005052       7        1   0.177827
27    -0.001534 -0.005052       8        1   0.177827
26    -0.001534 -0.005052       9        1   0.177827
25    -0.001534 -0.005052      10        1   0.177827
24    -0.001534 -0.005052      11        1   0.177827
23    -0.001534 -0.005052      12        1   0.177827
22    -0.001534 -0.005052      13        1   0.177827
20    -0.001534 -0.005052      14        1   0.177827
18    -0.001534 -0.005052      15        1   0.177827
17    -0.001534 -0.005052      16        1   0.177827
15    -0.001534 -0.005052      17        1   0.177827
1     -0.001534 -0.005052      18        1   0.177827
14    -0.001534 -0.005052      19        1   0.177827
13    -0.001534 -0.005052      20        1   0.177827
11    -0.001534 -0.005052      21        1   0.177827
10    -0.001534 -0.005052      22        1   0.177827
9     -0.001534 -0.005052      23        1   0.177827
8     -0.001534 -0.005052      24        1   0.177827
7     -0.001534 -0.005052      25        1   0.177827
5     -0.001534 -0.005052      26        1   0.177827
4     -0.001534 -0.005052      27        1   0.177827
3     -0.001534 -0.005052      28        1   0.177827
2     -0.001534 -0.005052      29        1   0.177827
29    -0.001534 -0.005052      30        1   0.177827, topic_info=        Term      Freq     Total Category  logprob  loglift
20      road  2.000000  2.000000  Default  30.0000  30.0000
5     driver  1.000000  1.000000  Default  29.0000  29.0000
12      list  1.000000  1.000000  Default  28.0000  28.0000
15    nation  1.000000  1.000000  Default  27.0000  27.0000
0   accident  1.000000  1.000000  Default  26.0000  26.0000
..       ...       ...       ...      ...      ...      ...
24   traffic  0.001994  1.070627  Topic30  -3.4965   0.0462
15    nation  0.001994  1.798244  Topic30  -3.4965  -0.4724
5     driver  0.001994  1.833724  Topic30  -3.4965  -0.4919
12      list  0.001994  1.833415  Topic30  -3.4965  -0.4918
20      road  0.001994  2.006085  Topic30  -3.4965  -0.5818

[1020 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
0         1  0.934032      accident
1         1  0.934032       alcohol
29        3  1.159038         asian
25        2  1.113379           bad
2         1  0.934032          base
3         1  0.934032         blood
26        2  1.111708       country
4         1  0.934032         death
5         1  0.545338        driver
5         2  0.545338        driver
6         1  0.934032          even
27        2  1.113360         first
30        3  1.159038        follow
7         1  0.934032         index
8         1  0.934032        injury
9         1  0.934032         legal
10        1  0.934032         level
11        1  0.934032         limit
12        1  0.545430          list
12        2  0.545430          list
13        1  0.934032          make
14        1  0.934032          more
15        1  0.556098        nation
15        3  0.556098        nation
16        1  0.934032  organisation
17        1  0.934032     parameter
28        2  1.113133         place
18        1  0.934032       quality
19        1  0.934032          rank
20        1  0.996967          road
31        3  1.159038     southeast
21        1  0.934032         speed
22        1  0.934032         study
23        1  0.934032          such
32        3  1.159038          then
24        1  0.934032       traffic, R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 13, 22, 7, 1, 17, 2